
<h1 align="center"><font size="5">How to Create Fake Data with Python</font></h1>

<h5 align="center"><font size="5">By:<br><br> Fatima, Sayeda </font></h5>

In [1]:
from IPython.display import HTML
def display_alert_color_1(message, color ,alert_type = "info"):
    display(HTML(f"<div class= 'alert alert-block alert-{alert_type}'> <H2> <span style='color:{color}'> {message} </span> </H2></div>"))
def display_alert_color_2(message, color ,alert_type = "info"):
    display(HTML(f"<div class= 'alert alert-block alert-{alert_type}'> <H3> <span style='color:{color}'> {message} </span> </H3></div>"))
from termcolor import colored     # To Display Colored Terminal Output
from datetime import datetime
script_start_time = (datetime.now()).strftime("%H:%M:%S")
display_alert_color_1("What the Process does?", "darkblue","success")
print(colored("The following process uses 'random' and 'barnum' python packages to create fake data ", 'green', attrs=['bold']))
display_alert_color_2("Get Script Start Time", "darkblue","info")
print(colored("Script Start Time: ", 'blue', attrs=['bold']) + colored(script_start_time, 'magenta', attrs=['bold']))
display_alert_color_2("Generate Fake Data", "darkmagenta","warning")

The following process uses 'random' and 'barnum' python packages to create fake data 


Script Start Time: 17:25:12


In [2]:
import random
import barnum
import pandas as pd
import re
import string
import datetime

# First create lists so that our function can restrict randomly generate data to assigned parameters
cost_centre_list = ["A001", "AB02", "CC28", "D456"]# contains strings representing cost centre codes.
profit_centre_list = ["00125", "23098", "23451", "00023", "87621"]# contains strings representing profit centre codes
currency_list = ["EUR", "USD", "INR", "KDD"]# contains strings representing different currency codes

# Define Function create_start_date that takes two optional parameters min_age and max_age with default values of 0.
def create_start_date(min_age=0, max_age=0):
    age = random.randint(min_age, max_age)# generates a random integer age between min_age and max_age inclusive
    start = datetime.date.today() - datetime.timedelta(days=random.randint(0, 365))# creates a random date within the past year by subtracting a random number of days from today's date. Finally, it subtracts age multiplied by 365 days from the randomly generated date to get a start date that is a random number of years in the past, between min_age and max_age inclusive.
    return start - datetime.timedelta(days=age * 365) 
start_date = create_start_date(min_age=1, max_age=5)# calls the function with arguments min_age=1 and max_age=5, and stores the returned value in a variable start_date
end_date = datetime.date.today()

def extract_data():
    # Use the barnum package to generate company data
    company_fields = [] # Create empty list to collect data
    email_suff = ["com", "org", "net", "edu"]
    for i in range(4): # creates 4 companies, increase the number as desired
        # Create a random company code with 2 capital letters + 2 numbers
        company_code = ''.join(random.choices(string.ascii_uppercase, k=2)) + ''.join(random.choices(string.digits, k=2))
        # Create Company name
        company_name = barnum.create_company_name(biz_type="Generic")
        # Create Company email
        ## Split the company name into words
        words = re.findall(r'\w+', company_name)
        ## Create email address using full company name without spaces + @ + first two words of company name + "email_suff"
        company_email = (words[0] + words[1].lower()) + "@" + words[0][0] + words[1][1] + "." + random.choice(email_suff)#suff
        data = {
            "company_name": company_name,
            "company_code": company_code,
            "city": barnum.create_city_state_zip(),
            "phone_number": barnum.create_phone(),
            "company_email": company_email
        }
        company_fields.append(data) # append created data to empty list "company_fields"
        company_data = pd.DataFrame(company_fields) # convert to pandas dataframe
        
        #### Now Generate Fake Invoice Data ####
        invoice_fields = [] # Create empty list to collect data
        for j in range(random.randint(100,200)): # Define minimum and maximum range
            first_name, last_name = barnum.create_name()
            name = first_name + " " + last_name
            invoice_value = random.randint(1000, 25000) # Define minimum and maximum values
            data = {
            "date": random.choice(pd.date_range(start=start_date, end=end_date).strftime("%Y-%m-%d")),
            "invoice": barnum.create_pw(),
            "text": barnum.create_sentence()[:10],
            "employee": name,
            "employee_email": last_name.lower() + "." + first_name[0].lower(),
            "cost_centre": random.choice(cost_centre_list),
            "profit_centre": random.choice(profit_centre_list),
            "currency": random.choice(currency_list),
            "value": invoice_value
            }
            invoice_fields.append(data) # append created data to empty list "invoice_fields"
        invoice_data = pd.DataFrame(invoice_fields) # convert to pandas dataframe
        invoice_data["date"] = pd.to_datetime(invoice_data["date"]) # date string to date object
        invoice_data["month"] = invoice_data["date"].dt.strftime("%b") # extract month from date object
        invoice_data["year"] = invoice_data["date"].dt.year # extract year from date object

        # Combine the company and invoice data
        complete_records = pd.concat([company_data] * len(invoice_data), ignore_index=True).merge(invoice_data, left_index=True, right_index=True, how='outer').sort_values(by=['company_code', 'date']).fillna("X", inplace=False)
        complete_records['employee_email'] = complete_records['employee_email']+"@"+(complete_records['company_email'].str.split('@').str[1])
        #print(list(complete_records.columns))# you may uncomment this line to get all column names as list and then rearrange or delete the names you don't want in the "final_columns" list below
        final_columns = ['company_code','company_name', 'city', 'company_email', 'phone_number', 'year', 'month', 'date', 'invoice', 'text', 'employee', 'employee_email', 'cost_centre', 'profit_centre', 'currency', 'value']
        complete_records = complete_records[complete_records["invoice"] != "X"].reset_index(drop=True).reindex(columns=final_columns)
    return complete_records

complete_records = extract_data() # run the function above and return dataframe with fake data
complete_records
#print(complete_records)

,company_code,company_name,city,company_email,phone_number,year,month,date,invoice,text,employee,employee_email,cost_centre,profit_centre,currency,value
0,BN71,Construction Software Vision Co,"(94953, Petaluma, CA)",Constructionsoftware@Co.net,(646)772-3780,2019,Oct,2019-10-17 00:00:00,3OUaR6wh,Commodo co,Johnathan Rees,rees.j@Co.net,CC28,00023,USD,21211
1,BN71,Construction Software Vision Co,"(94953, Petaluma, CA)",Constructionsoftware@Co.net,(646)772-3780,2019,Nov,2019-11-12 00:00:00,0ZXyqo0C,Feugait in,Destiny Horne,horne.d@Co.net,AB02,00125,INR,4400
2,BN71,Construction Software Vision Co,"(94953, Petaluma, CA)",Constructionsoftware@Co.net,(646)772-3780,2020,Feb,2020-02-16 00:00:00,0QfBrTf7,Ex duis te,Nigel Etheridge,etheridge.n@Co.net,A001,00023,KDD,19118
3,BN71,Construction Software Vision Co,"(94953, Petaluma, CA)",Constructionsoftware@Co.net,(646)772-3780,2020,Apr,2020-04-27 00:00:00,2aZEaU4d,Facilisi d,Mickey Vanwinkle,vanwinkle.m@Co.net,AB02,00125,KDD,6511
4,BN71,Construction Software Vision Co,"(94953, Petaluma, CA)",Constructionsoftware@Co.net,(646)772-3780,2020,May,2020-05-04 00:00:00,z4YSqa1s,Ipsum mole,Ruthie Pride,pride.r@Co.net,CC28,23098,EUR,10574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,XQ49,Building Contract Solutions Group,"(95220, Acampo, CA)",Buildingcontract@Bo.net,(719)827-3552,2022,Jun,2022-06-19 00:00:00,Gdk9w2Rk,Duis autem,Brigitte Goldberg,goldberg.b@Bo.net,D456,00023,USD,16256
101,XQ49,Building Contract Solutions Group,"(95220, Acampo, CA)",Buildingcontract@Bo.net,(719)827-3552,2022,Aug,2022-08-02 00:00:00,N9bq2mUF,Eu veniam,Eloy Langer,langer.e@Bo.net,CC28,23098,USD,21246
102,XQ49,Building Contract Solutions Group,"(95220, Acampo, CA)",Buildingcontract@Bo.net,(719)827-3552,2022,Sep,2022-09-07 00:00:00,L9Wl6Ubl,Ut esse do,Amparo Sipes,sipes.a@Bo.net,CC28,87621,EUR,13098
103,XQ49,Building Contract Solutions Group,"(95220, Acampo, CA)",Buildingcontract@Bo.net,(719)827-3552,2023,Jan,2023-01-23 00:00:00,a1gVFp1e,Nonummy ad,Malcolm Truitt,truitt.m@Bo.net,A001,23451,USD,23474


In [3]:
display_alert_color_2("Summarize Pivot Table", "darkmagenta","warning")
print(colored("We will create a simple pivot table to summarize invoice count and total value by company.", 'green', attrs=['bold']))

We will create a simple pivot table to summarize invoice count and total value by company.


In [4]:
pivot_table_1 = complete_records.pivot_table(index='company_name', values=['invoice', 'value'], aggfunc={'invoice':'count'})# groups data by "company_name" column and aggregates "invoice" column by counting the number of occurrences and the "value" column by summing up the values
pivot_table_1.reset_index(inplace=True) #moves the index to a new column named company_name using the reset_index method and 
pivot_table_1.index += 1 # sets the index to start from 1 by adding 1 to each index value.
pivot_table_1

,company_name,invoice
1,Building Contract Solutions Group,27
2,Construction Software Vision Co,26
3,Contract Incorporated,26
4,Interactive Federated LLC,26


In [5]:
pivot_table_2 = complete_records.pivot_table(index=['company_name', 'currency'], values=['invoice', 'value'], aggfunc={'invoice':'count', 'value':'sum'})# groups data pivot_table_1.reset_index(inplace=True) #moves the index to a new column named company_name using the reset_index method and 
pivot_table_2.reset_index(level=[0,1], inplace=True)# moves the index to new columns named 'company_name' and 'currency' using the reset_index method and 
pivot_table_2

,company_name,currency,invoice,value
0,Building Contract Solutions Group,EUR,7,91035.0
1,Building Contract Solutions Group,INR,3,47052.0
2,Building Contract Solutions Group,KDD,7,100554.0
3,Building Contract Solutions Group,USD,10,160046.0
4,Construction Software Vision Co,EUR,3,23850.0
5,Construction Software Vision Co,INR,8,105514.0
6,Construction Software Vision Co,KDD,8,112560.0
7,Construction Software Vision Co,USD,7,78144.0
8,Contract Incorporated,EUR,9,115736.0
9,Contract Incorporated,INR,6,58280.0


In [6]:
from datetime import datetime # we need to import this module again since it was overwritten when we "import datetime" 
script_end_time = (datetime.now()).strftime("%H:%M:%S")
# convert time string to datetime
t1 = datetime.strptime(script_start_time, "%H:%M:%S")
t2 = datetime.strptime(script_end_time, "%H:%M:%S")
delta = t2- t1
sec = delta.total_seconds()# get difference in seconds
min = (round(sec/60))# get difference in minutes
hours = round(sec / (60 * 60)) # get difference in hours
display_alert_color_2("Get Script Start Time", "darkblue","info")
print(colored("Script End Time: ", 'blue', attrs=['bold']) + colored((script_end_time), 'magenta', attrs=['bold'])
     + colored("\n\nTotal Time Taken: ", 'green', attrs=['bold']) + colored((t2-t1), 'red', attrs=['bold'])
      + colored("\nSeconds: ", 'green', attrs=['bold']) + colored(sec, 'red', attrs=['bold'])
      + colored("\nMinutes: ", 'green', attrs=['bold']) + colored(min, 'red', attrs=['bold'])
      + colored("\nHours: ", 'green', attrs=['bold']) + colored(hours, 'red', attrs=['bold']))

Script End Time: 17:25:17

Total Time Taken: 0:00:05
Seconds: 5.0
Minutes: 0
Hours: 0
